In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import gc

In [4]:
dtypes = {
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'hour'          : 'uint8',
        'is_attributed' : 'uint8', 
        'ip_day_hour_count': 'uint32', 
        'ip_os_day_hour_count': 'uint32', 
        'ip_app_day_hour_count': 'uint32', 
        'ip_app_os_day_hour_count': 'uint32', 
        'app_day_hour_count': 'uint32', 
        'ip_device_os_count': 'uint32', 
        'ip_app_device_os_count': 'uint32', 
        'ip_device_os_mean': 'float16',
        'ip_app_device_os_mean': 'float16',
        'ip_app_device_mean': 'float16',
        'app_device_os_mean': 'float16',
        'ip_device_os_time2nextclick': 'int32',
        'ip_app_device_os_time2nextclick': 'int32',
        'ip_app_device_time2nextclick': 'int32',
        'ip_device_os_time2previousclick': 'int32',
        'ip_app_device_os_time2previousclick': 'int32',
        'ip_app_device_time2previousclick': 'int32',
        'ip_device_os_countfromfuture': 'uint32', 
        'ip_app_device_os_countfromfuture': 'uint32', 
        'ip_app_device_countfromfuture': 'uint32', 
        'ip_device_os_countfrompast': 'uint32', 
        'ip_app_device_os_countfrompast': 'uint32', 
        'ip_app_device_countfrompast': 'uint32', 
        'ip_device_os_lasttimediff': 'int32',
        'ip_app_device_os_lasttimediff': 'int32',
        'ip_app_device_lasttimediff': 'int32',
        'ip_device_os_firsttimediff': 'int32',
        'ip_app_device_os_firsttimediff': 'int32',
        'ip_app_device_firsttimediff': 'int32',
        'matrixFact_user_iposdeviceapp_item_app': 'float16',
        'matrixFact_user_ip_item_appdeviceos': 'float16',
        'matrixFact_user_ipchannel_item_appdeviceos': 'float16',
        'ip_device_os_regression': 'float16',
        'ip_app_device_os_regression': 'float16',
        'ip_app_device_regression': 'float16',
        'ip_app_device_os_channel_regression': 'float16', 
        'attributed_timediffmax':'int32',
        'attributed_timediffmin':'int32',
        'attributed_timediff':'float16',
        'matrixFact_user_ipappdeviceos_item_channel': 'float16'
    
        } 

In [5]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
file_format = '{}_features_supplementV2_feature41.csv'
day_list = ['day7', 'day8', 'day9']
df_dict = {}
df_extra = {}
for file in ['day7', 'day8', 'day9']: 
    df_dict[file] = pd.read_csv(load_path+file_format.format(file),dtype=dtypes)
    df_extra[file] = pd.read_csv(load_path+'{}_features_supplementV2_extra_timediff.csv'.format(file),dtype=dtypes)
    print(file_format.format(file))
df_dict['test'] = pd.read_csv(load_path+file_format.format('test_equalhour'),dtype=dtypes)
df_extra['test'] =  pd.read_csv(load_path+'test_equalhoursV2_features_supplementV2_extra_timediff.csv',dtype=dtypes)

print(file_format.format('test_equalhour'))
    

day7_features_supplementV2_feature41.csv
day8_features_supplementV2_feature41.csv
day9_features_supplementV2_feature41.csv
test_equalhour_features_supplementV2_feature41.csv


In [6]:
for day in ['day7', 'day8', 'day9', 'test']:
#     for col in ['matrixFact_user_ipappdeviceos_item_channel', 'attributed_timediff']:
    for col in [ 'attributed_timediff']:
        df_dict[day][col] = df_extra[day][col]
        print(col)

attributed_timediff
attributed_timediff
attributed_timediff
attributed_timediff


In [7]:
del df_extra 
gc.collect()

221

# define parameter

In [8]:
target = 'is_attributed'

day_list = ['day7', 'day8', 'day9']

combine = 0
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 4000,
        'learning_rate': 0.1,
        'num_leaves': 61,
        'num_threads': 4, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 390,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.7, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':60,
        'bagging_fraction': 0.7, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        'verbose': 0,
        'scale_pos_weight': 400,
        'metric' : [ 'auc']
    }

categorical_col = [ 'app', 'device', 'os', 'channel', 'hour']
feature_cols = list(df_dict['test'].columns.values)

In [9]:
df_dict['day7'].columns.values

array(['app', 'device', 'os', 'channel', 'hour', 'is_attributed',
       'ip_day_hour_count', 'ip_os_day_hour_count',
       'ip_app_day_hour_count', 'ip_app_os_day_hour_count',
       'app_day_hour_count', 'ip_device_os_count',
       'ip_app_device_os_count', 'ip_device_os_mean',
       'ip_app_device_os_mean', 'ip_app_device_mean', 'app_device_os_mean',
       'ip_device_os_time2nextclick', 'ip_app_device_os_time2nextclick',
       'ip_app_device_time2nextclick', 'ip_device_os_time2previousclick',
       'ip_app_device_os_time2previousclick',
       'ip_app_device_time2previousclick', 'ip_device_os_countfromfuture',
       'ip_app_device_os_countfromfuture', 'ip_app_device_countfromfuture',
       'ip_device_os_countfrompast', 'ip_app_device_os_countfrompast',
       'ip_app_device_countfrompast', 'ip_device_os_lasttimediff',
       'ip_app_device_os_lasttimediff', 'ip_app_device_lasttimediff',
       'ip_device_os_firsttimediff', 'ip_app_device_os_firsttimediff',
       'ip_app_dev

In [10]:
# get rid of extra features
# extra = set(['matrixFact_user_iposdeviceapp_item_channel', 'matrixFact_user_iposdeviceapp_item_device'])
extra = set(['ip_device_os_countself',
             'ip_day_hour_countself', 
             'app_day_hour_countself',
             'ip_app_device_os_countself',
             'ip_app_day_hour_countself', 
             'ip_os_day_hour_countself', 
             'ip_app_os_day_hour_countself',
             'ip_device_os_regression',
             'ip_app_device_os_regression', 
             'ip_app_device_regression' ])

# extra = set([])
# extra = set(['ip_device_os_count',
#              'ip_day_hour_count', 
#              'app_day_hour_count',
#              'ip_app_device_os_count',
#              'ip_app_day_hour_count', 
#              'ip_os_day_hour_count', 
#              'ip_app_os_day_hour_count' ])
feature_cols = list(set(feature_cols) - extra)

In [11]:
feature_cols

['matrixFact_user_iposdeviceapp_item_app',
 'ip_app_os_day_hour_count',
 'ip_app_device_firsttimediff',
 'ip_os_day_hour_count',
 'ip_app_device_os_firsttimediff',
 'ip_device_os_time2previousclick',
 'ip_device_os_firsttimediff',
 'ip_device_os_countfrompast',
 'device',
 'ip_app_device_os_channel_regression',
 'ip_app_device_os_count',
 'ip_app_device_countfrompast',
 'ip_app_day_hour_count',
 'attributed_timediff',
 'app_device_os_mean',
 'app',
 'ip_app_device_os_countfromfuture',
 'ip_device_os_count',
 'ip_app_device_os_time2nextclick',
 'ip_app_device_time2previousclick',
 'channel',
 'ip_app_device_lasttimediff',
 'ip_device_os_mean',
 'ip_app_device_mean',
 'ip_app_device_os_countfrompast',
 'ip_day_hour_count',
 'ip_app_device_countfromfuture',
 'ip_app_device_os_mean',
 'os',
 'matrixFact_user_ipchannel_item_appdeviceos',
 'ip_app_device_os_time2previousclick',
 'app_day_hour_count',
 'ip_device_os_lasttimediff',
 'ip_device_os_time2nextclick',
 'ip_app_device_time2nextclick

In [ ]:
def train_lightgbm(x_train, x_val, feature_cols, categorical_feature, params, best_round = None, target='is_attributed'):
    param = params.copy()
    y_train = x_train[target].values
    y_val = x_val[target].values
    
    lgb_train = lgb.Dataset(x_train[feature_cols], y_train, categorical_feature = categorical_feature)
    lgb_val = lgb.Dataset(x_val[feature_cols], y_val, categorical_feature = categorical_feature)
    if best_round is not None:
        param['num_rounds'] = best_round
        del param['early_stopping_round']
    print('start training')
    model = lgb.train(param, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)
    return model

In [ ]:
for day in day_list:
# for day in ['day7', 'day8']:
    train_day = list(set(day_list)-set([day]))
    file_name = 'concat_{}{}_val_{}_allday_supplement_V2_attributediff_noreg.npy'.format(train_day[0],train_day[1],day)
    print(file_name)
    trainset = pd.concat([df_dict[train_day[0]],df_dict[train_day[1]]])
    valset = df_dict[day]
    print('building train val done!')
    model = train_lightgbm(trainset, valset, feature_cols, categorical_col, params)
    best_round = model.best_iteration
    
    df_all = pd.concat([trainset, valset])
    del trainset
    gc.collect()
    model = train_lightgbm(df_all, valset, feature_cols, categorical_col, params, best_round)
    del df_all
    gc.collect()
    
    pred = model.predict(df_dict['test'][feature_cols])
    np.save(load_path+file_name, pred)
    
    # prediction
    df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
    print('loading file done!')
    df_sub = pd.DataFrame()
    df_sub['click_id'] = df_test_raw['click_id']
    df_sub['is_attributed'] = pred
    print('predicting file done!')
    df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/concat_{}{}_val_{}_allday_supplement_V2_bestround_{}_attributediff_noreg.csv.gz'.format(train_day[0],train_day[1],day,best_round), compression='gzip', index=False)

    

concat_day8day9_val_day7_allday_supplement_V2_attributediff_noreg.npy
building train val done!
start training


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 60 rounds.
[10]	valid_0's auc: 0.980032
[20]	valid_0's auc: 0.982183
[30]	valid_0's auc: 0.983498
[40]	valid_0's auc: 0.984361
[50]	valid_0's auc: 0.984953
[60]	valid_0's auc: 0.985316
[70]	valid_0's auc: 0.98552
[80]	valid_0's auc: 0.98566
[90]	valid_0's auc: 0.985711
[100]	valid_0's auc: 0.985777
[110]	valid_0's auc: 0.985825
[120]	valid_0's auc: 0.985838
[130]	valid_0's auc: 0.985844
[140]	valid_0's auc: 0.985863
[150]	valid_0's auc: 0.98587
[100]	valid_0's auc: 0.986007
[110]	valid_0's auc: 0.986059
[120]	valid_0's auc: 0.98607
[130]	valid_0's auc: 0.986079
[140]	valid_0's auc: 0.986096
[150]	valid_0's auc: 0.986092
[160]	valid_0's auc: 0.986075
[170]	valid_0's auc: 0.986075
[180]	valid_0's auc: 0.986081
[190]	valid_0's auc: 0.986075
[200]	valid_0's auc: 0.986069
Early stopping, best iteration is:
[144]	valid_0's auc: 0.9861


In [16]:
importance = pd.Series(model.feature_importance(importance_type='gain'), index=feature_cols)
# importance = pd.Series(model.feature_importance(), index=feature_cols)
importance = importance.sort_values(ascending=False)
if len(model.feature_importance()) != len(feature_cols):
    raise ValueError('Feature importance has length: {}, \n while feature number is {}'.
                     format(len(model.feature_importance()), len(feature_cols)))
    
importance

matrixFact_user_iposdeviceapp_item_app        7.855351e+08
app_device_os_mean                            2.552609e+08
matrixFact_user_ip_item_appdeviceos           1.608068e+08
matrixFact_user_ipchannel_item_appdeviceos    6.137858e+07
ip_app_device_os_time2nextclick               4.482689e+07
channel                                       2.797751e+07
ip_day_hour_count                             2.198434e+07
app                                           2.149862e+07
os                                            1.951830e+07
ip_app_device_os_lasttimediff                 1.507538e+07
ip_device_os_count                            1.480736e+07
hour                                          1.057946e+07
ip_app_day_hour_count                         6.929483e+06
device                                        6.445461e+06
ip_app_os_day_hour_count                      6.426826e+06
ip_app_device_os_count                        5.928128e+06
ip_app_device_time2nextclick                  4.825258e+

# Average

In [14]:
#### load each and average
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
day7 = np.load(load_path+'concat_day8day9_val_day7_allday_supplement_V2_attributediff_noreg.npy')
day8 = np.load(load_path+'concat_day7day9_val_day8_allday_supplement_V2_attributediff_noreg.npy')
day9 = np.load(load_path+'concat_day7day8_val_day9_allday_supplement_V2_attributediff_noreg.npy')

pred = (day7 + day8 + day9)/3

In [15]:
# prediction
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
print('loading file done!')
df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']
df_sub['is_attributed'] = pred
print('predicting file done!')
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/allday_blend_supplement_V2_attributediff_noreg.csv.gz', compression='gzip', index=False)


loading file done!
predicting file done!
